In [6]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the input shape and number of classes
INPUT_SHAPE = (128, 128, 3)  # Assume 128x128 RGB images
NUM_CLASSES = 2  # Food or not food

# Set the batch size
BATCH_SIZE = 64

# Create an ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

# Load and preprocess the food images
def load_and_preprocess_images():
  # Create a dataset from the 'food_images' directory
  dataset = tf.keras.preprocessing.image_dataset_from_directory(
      'food_images_2',
      validation_split=0.2,
      subset='training',
      seed=123,
      image_size=INPUT_SHAPE[:2],
      batch_size=BATCH_SIZE
  )

  # Use the ImageDataGenerator to augment and preprocess the images
  def preprocess(image, label):
    image = datagen(image)
    return image, label

  dataset = dataset.map(preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  dataset = dataset.cache().prefetch(tf.data.experimental.AUTOTUNE)

  # Split the dataset into training and validation sets
  x_train, y_train = dataset.take(-int(0.2*len(dataset)))
  x_val, y_val = dataset.skip(-int(0.2*len(dataset)))

  # Return the dataset as a tuple of NumPy arrays
  return (x_train.numpy(), y_train.numpy()), (x_val.numpy(), y_val.numpy())

# Load and preprocess the food images
(x_train, y_train), (x_val, y_val) = load_and_preprocess_images()

# Define the encoder
encoder_input = layers.Input(shape=INPUT_SHAPE)
x = layers.Conv2D(16, 3, activation='relu', padding='same')(encoder_input)
x = layers.MaxPooling2D(2, padding='same')(x)
x = layers.Conv2D(8, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D(2, padding='same')(x)
x = layers.Conv2D(8, 3, activation='relu', padding='same')(x)
encoder_output = layers.MaxPooling2D(2, padding='same')(x)

# Define the decoder
decoder_input = layers.Input(shape=K.int_shape(encoder_output)[1:])
x = layers.Conv2D(8, 3, activation='relu', padding='same')(decoder_input)
x = layers.UpSampling2D(2)(x)
x = layers.Conv2D(8, 3, activation='relu', padding='same')(x)
x = layers.UpSampling2D(2)(x)
x = layers.Conv2D(16, 3, activation='relu')(x)
x = layers.UpSampling2D(2)(x)
decoder_output = layers.Conv2D(3, 3, activation='sigmoid', padding='same')(x)

# Define the autoencoder
autoencoder = tf.keras.Model(encoder_input, decoder_output)

# Define the encoder model
encoder = tf.keras.Model(encoder_input, encoder_output)

# Compile the autoencoder with a loss function and an optimizer
autoencoder.compile(loss='binary_crossentropy', optimizer='adam')

# Train the autoencoder
autoencoder.fit(x_train, x_train,
                epochs=10,
                batch_size=64,
                validation_data=(x_val, x_val))

# Generate latent representations of the food images using the encoder
latent_representations = encoder.predict(x_train)


# Train a classifier on the latent representations
classifier = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=K.int_shape(latent_representations)[1:]),
    layers.Dense(NUM_CLASSES, activation='softmax')
])
classifier.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
classifier.fit(latent_representations, y_train, epochs=10, batch_size=64)

# Evaluate the classifier on the validation set
loss, accuracy = classifier.evaluate(encoder.predict(x_val), y_val)
print('Validation loss:', loss)
print('Validation accuracy:', accuracy)

# Classify a new image as food or not food
#new_image = preprocess_image(...)  # Preprocess the new image
#predictions = classifier.predict(encoder.predict(new_image))
#predicted_class = np.argmax(predictions)
#print('Predicted class:', predicted_class)  # 0 for food, 1 for not food




Found 0 files belonging to 0 classes.
Using 0 files for training.


ValueError: No images found in directory food_images_2. Allowed formats: ('.bmp', '.gif', '.jpeg', '.jpg', '.png')